In [2]:
#import library
import pandas as pd
import json
from textblob import TextBlob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [3]:
#read json file into dataframe [assignable,category,category_id]
f = open("US_category_id.json")
data = f.read()
f.close()
out = json.loads(data)
categories = pd.DataFrame([
    {'category_id':x['id'],
     'category':x['snippet']['title'],
     'assignable':x['snippet']['assignable']
    } for x in out['items']
])

In [4]:
categories.head()

,category_id,category,assignable
0,1,Film & Animation,True
1,2,Autos & Vehicles,True
2,10,Music,True
3,15,Pets & Animals,True
4,17,Sports,True


In [5]:
#change category_id to integer
categories['category_id'] = categories['category_id'].astype(int)

In [12]:
#read in US video and US comment and change the category_id to integer
USvideos = pd.read_csv('USvideos.csv',error_bad_lines=False)
UScomments = pd.read_csv('UScomments.csv',error_bad_lines=False)
USvideos['category_id'] = USvideos['category_id'].astype(int)

b'Skipping line 2401: expected 11 fields, saw 21\nSkipping line 2800: expected 11 fields, saw 21\nSkipping line 5297: expected 11 fields, saw 12\nSkipping line 5299: expected 11 fields, saw 12\nSkipping line 5300: expected 11 fields, saw 12\nSkipping line 5301: expected 11 fields, saw 12\n'
b'Skipping line 41589: expected 4 fields, saw 11\nSkipping line 51628: expected 4 fields, saw 7\nSkipping line 114465: expected 4 fields, saw 5\n'
b'Skipping line 142496: expected 4 fields, saw 8\nSkipping line 189732: expected 4 fields, saw 6\nSkipping line 245218: expected 4 fields, saw 7\n'
b'Skipping line 388430: expected 4 fields, saw 5\n'


In [13]:
videos.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,category_name,likes_log,views_log,dislikes_log,comment_log,like_rate,dislike_rate,comment_rate
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,Entertainment,12.676245,15.295757,8.688117,10.741730,7.283816,0.134979,1.052451
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09,Science & Technology,12.132717,15.877312,10.191670,0.000000,2.364506,0.339422,0.000000
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09,People & Blogs,13.264901,15.581253,10.590994,12.047716,9.863257,0.680373,2.920128
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09,Science & Technology,10.125671,14.787086,8.421343,9.459541,0.945270,0.171909,0.485560
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09,Comedy,11.479027,13.970916,6.343880,8.804925,8.275278,0.048625,0.570656


In [14]:
#assign format to date
videos['date'] =  pd.to_datetime(USvideos.date, format="%d.%m")

ValueError: time data '13' does not match format '%d.%m' (match)

In [8]:
#Merged the categories in 
USvideos=USvideos.merge(categories, on='category_id')

In [9]:
#Using Textblob library to calculate sentiments
def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return None

In [10]:
#add one more column of [polarity] to UScomments
UScomments['polarity']=UScomments['comment_text'].apply(sentiment_calc)

In [11]:
#function to change the continuous sentiment score to discrete
def pol2sent(pol):
    if pol<=-.5:
        return "Very negative"
    if pol>-.5 and pol<0:
        return "Negative"
    if pol == 0:
        return "Neutral"
    if pol > 0 and pol<0.5:
        return "Positive"
    if pol>=0.5:
        return "Very positive"

In [12]:
def grouped_weighted_avg(values, weights, by):
    return (values * weights).groupby(by).sum() / weights.groupby(by).sum()

In [13]:
#create a video sentiment data frame with [polarity,video_id] and video_id index
Videosentiment = UScomments.groupby("video_id").mean()
Videosentiment['video_id'] = Videosentiment.index
Videosentiment.head()

,polarity,video_id
video_id,,
--JinobXWPk,0.046776,--JinobXWPk
-1fzGnFwz9M,0.137792,-1fzGnFwz9M
-3AGlBYyLjo,-0.306250,-3AGlBYyLjo
-5sCWsLlTCI,0.096262,-5sCWsLlTCI
-6Zc8Co2H3w,0.230334,-6Zc8Co2H3w


In [14]:
#add one more column [polarity] by merging USvideos and Videosentiment on video_id
USvideos=USvideos.merge(Videosentiment, on="video_id")

In [15]:
#add one more column [videosentiment] that change the video polarity to discrete
USvideos['videosentiment']=USvideos['polarity'].apply(pol2sent)

In [1]:
USvideos.head(300)

NameError: name 'USvideos' is not defined

In [17]:
#repeat the same for GBvideos
GBf = open("GB_category_id.json")
data = GBf.read()
GBf.close()
GBout = json.loads(data)
GBcategories = pd.DataFrame([
    {'category_id':x['id'],
     'category':x['snippet']['title'],
     'assignable':x['snippet']['assignable']
    } for x in GBout['items']
])

In [18]:
GBcategories.head()

,assignable,category,category_id
0,True,Film & Animation,1
1,True,Autos & Vehicles,2
2,True,Music,10
3,True,Pets & Animals,15
4,True,Sports,17


In [19]:
#change category_id to integer
GBcategories['category_id'] = GBcategories['category_id'].astype(int)

In [20]:
#read in GB video and US comment and change the category_id to integer
GBvideos = pd.read_csv('GBvideos.csv',error_bad_lines=False)
GBcomments = pd.read_csv('GBcomments.csv',error_bad_lines=False)
GBvideos['category_id'] = USvideos['category_id'].astype(int)

b'Skipping line 2398: expected 11 fields, saw 21\nSkipping line 2797: expected 11 fields, saw 21\n'
b'Skipping line 113225: expected 4 fields, saw 5\n'
b'Skipping line 158379: expected 4 fields, saw 7\nSkipping line 241590: expected 4 fields, saw 5\nSkipping line 245637: expected 4 fields, saw 7\n'
b'Skipping line 521402: expected 4 fields, saw 5\n'


In [21]:
GBvideos.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,24.0,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240,13548,705,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,13.09
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,This Morning,24.0,this morning|interview|holly willoughby|philli...,494203,2651,1309,0,https://i.ytimg.com/vi/AqokkXoa7uE/default.jpg,13.09
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,emmablackery,24.0,emmablackery|emma blackery|emma|blackery|briti...,142819,13119,151,1141,https://i.ytimg.com/vi/YPVcg45W0z4/default.jpg,13.09
3,T_PuZBdT2iM,getting into a conversation in a language you ...,ProZD,24.0,skit|korean|language|conversation|esl|japanese...,1580028,65729,1529,3598,https://i.ytimg.com/vi/T_PuZBdT2iM/default.jpg,13.09
4,NsjsmgmbCfc,Baby Name Challenge!,Sprinkleofglitter,24.0,sprinkleofglitter|sprinkle of glitter|baby gli...,40592,5019,57,490,https://i.ytimg.com/vi/NsjsmgmbCfc/default.jpg,13.09


In [22]:
#assign format to date
GBvideos['date'] =  pd.to_datetime(GBvideos.date, format="%d.%m")

In [23]:
#Merged the categories in 
GBvideos=GBvideos.merge(categories, on='category_id')

In [24]:
#add one more column of [polarity] to UScomments
GBcomments['polarity']=GBcomments['comment_text'].apply(sentiment_calc)

In [25]:
#create a video sentiment data frame with [polarity,video_id] and video_id index
GBVideosentiment = GBcomments.groupby("video_id").mean()
GBVideosentiment['video_id'] = GBVideosentiment.index
GBVideosentiment.head()

,likes,replies,polarity,video_id
video_id,,,,
--JinobXWPk,0.200000,0.013333,0.037737,--JinobXWPk
--y69Ui8xwM,3.872852,0.989691,0.170290,--y69Ui8xwM
-1fzGnFwz9M,8.750000,0.205000,0.136542,-1fzGnFwz9M
-2hRYEFiPSA,1.457143,0.092857,0.239867,-2hRYEFiPSA
-6Zc8Co2H3w,0.110000,0.003750,0.205802,-6Zc8Co2H3w


In [26]:
GBVideosentiment=GBVideosentiment[['polarity','video_id']]
GBVideosentiment.head()

,polarity,video_id
video_id,,
--JinobXWPk,0.037737,--JinobXWPk
--y69Ui8xwM,0.170290,--y69Ui8xwM
-1fzGnFwz9M,0.136542,-1fzGnFwz9M
-2hRYEFiPSA,0.239867,-2hRYEFiPSA
-6Zc8Co2H3w,0.205802,-6Zc8Co2H3w


In [35]:
#add one more column [polarity] by merging GBvideos and GBVideosentiment on video_id
GBvideos=GBvideos.merge(GBVideosentiment, on="video_id")

In [37]:
#add one more column [videosentiment] that change the video polarity to discrete
GBvideos['videosentiment']=GBvideos['polarity'].apply(pol2sent)

In [38]:
GBvideos.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,assignable,category,polarity,videosentiment
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,24,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240,13548,705,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,1900-09-13,True,Entertainment,0.078579,Positive
1,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,24,apple events|apple event|iphone 8|iphone x|iph...,7920718,81587,14433,1460,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,1900-09-14,True,Entertainment,0.078579,Positive
2,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,24,apple events|apple event|iphone 8|iphone x|iph...,8260919,84514,15178,2049,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,1900-09-15,True,Entertainment,0.078579,Positive
3,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,24,apple events|apple event|iphone 8|iphone x|iph...,8443106,86006,15504,2381,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,1900-09-16,True,Entertainment,0.078579,Positive
4,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,emmablackery,24,emmablackery|emma blackery|emma|blackery|briti...,142819,13119,151,1141,https://i.ytimg.com/vi/YPVcg45W0z4/default.jpg,1900-09-13,True,Entertainment,0.142932,Positive


In [39]:
videos = pd.concat([USvideos, GBvideos]).reset_index()
comments = pd.concat([UScomments, GBcomments]).reset_index()

In [40]:
videos.to_csv("US_GB_videos_after_sentiment_analysis.csv")
comments.to_csv("US_GB_comments_after_sentiment_analysis.csv")